### 볼린저 밴드 함수화
1. 첫 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 결측치와 이상치를 제거한다.
    2. 컬럼을 종가만 두고 모두 삭제
    3. 이동평균선, 상단밴드, 하단밴드 파생변수 생성
2. 두 번째 함수를 생성(매개변수, 데이터프레임 하나)
    1. 거래 내역이라는 파생 변수 생성 값은 ""
    2. 볼린저 밴드의 거래 내역을 추가
    3. 거래 내역을 출력(print 사용)
3. 세 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 수익률 파생변수 생성 값은 "1"
    2. 판매를 한 날의 수익을 변경
    3. 누적 수익률을 계산하여 새로운 파생변수를 생성
    4. 최종 누적 수익률을 출력(print 사용)

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf

In [5]:
# 1번 함수 생성
def boll(df,col, start, end):
    #인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index)
    # start, end를 시계열로 변경
    start = datetime.strptime(start, '%Y%m%d').isoformat()
    end = datetime.strptime(end, '%Y%m%d').isoformat()
    #데이터를 시작시간부터 종료시간까지 필터
    df =df.loc[start:end]
    #결측치 제거
    df = df.loc[~df.isin((np.nan, np.inf, -np.inf)).any(1),['Adj Close']]
    # price_df = df[['Adj Close']] 위에 문장이랑 합침 위에는 인덱스 명령어만 잇었기 때문 
    df['center'] = df.rolling(20).mean() # ['Adj Close'] 생략략
    df['lb'] = df['center']-(2*df[col].rolling(20).std())
    df['ub'] = df['center']+(2*df[col].rolling(20).std())
    return df

In [6]:
data = pd.read_csv("../csv/GM.csv",index_col='Date')

In [7]:
data = boll(data, 'Adj Close',  '20110101', '20150101')

C:\Users\bjw\AppData\Local\Temp\ipykernel_25480\1284579565.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df.loc[~df.isin((np.nan, np.inf, -np.inf)).any(1),['Adj Close']]


In [8]:
def boll2(df):
    # 기준이 되는 컬럼이 무엇인가?
    # 기준이 되는 컬럼은 컬럼 중에 첫번째이기 때문에 df.columns[0]
    col= df.columns[0]
    df['trade']=""
    for i in df.index:
        # 상단밴드보다 종가가 높은 경우
        if df.loc[i,col]> df.loc[i,'ub']:
        # 현재 구매상태이면
            if df.shift(1).loc[i,'trade'] == 'buy':
            # 매도
                df.loc[i,'trade']= ''
            else:
            #구매상태가아니면
                df.loc[i,'trade']= ''
        # 하단밴드보다 종가가 낮은경우
        elif df.loc[i,col]< df.loc[i,'lb']:
        # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
            #구매상태 유지
                df.loc[i,'trade'] ='buy'
            else:
                df.loc[i,'trade'] ='buy'
        else:
        #현재 구매상태이면
            if df.shift(1).loc[i,'trade'] =='buy':
            #구매 상태유지
                df.loc[i,'trade']= 'buy'
            else:
                df.loc[i,'trade'] =''
    return df

In [9]:
boll2(data)['trade'].value_counts()

buy    547
       459
Name: trade, dtype: int64

In [10]:
data = boll2(data)

In [11]:
def boll_rtn(df):
    col =df.columns[0]
    df['return'] =1
    rtn=1.0
    buy=0.0
    sell=0.0

    for i in df.index:
    # 구매가를 출력
        if (df.shift(1).loc[i,'trade']=='')and(df.loc[i,'trade']=='buy'):
            buy = df.loc[i,col]
            print('진입일:',i,'구매가격:', buy)
    #판매가 출력
        elif(df.shift(1).loc[i,'trade']=='buy')and(df.loc[i,'trade']==''):
            sell= df.loc[i,col]
            rtn = (sell-buy)/buy+1
            df.loc[i,'return'] =rtn
            print('판매일:',i,'판매가격:', sell, '수익률:' , rtn)

    #구매가, 판매가를 초기화
        if df.loc[i,'trade']=='':
            buy=0.0
            sell=0.0

    acc_rtn = 1.0
    for i in df.index:
        rtn=df.loc[i,'return']
        acc_rtn*=rtn
        df.loc[i,'acc_rtn']=acc_rtn

    print('누적수익률:', acc_rtn)
    
    return df
    


In [ ]:
boll_rtn(data, 'Adj Close')

In [ ]:
import bollinger as bg
import imp
imp.reload(bg)

In [ ]:
df = pd.read_csv("../csv/MSFT.csv", index_col='Date')

In [ ]:
# 1번 함수 실행
data = bg.boll(df,'Adj Close','20100101','20190101')

In [ ]:
# 2번 함수 실행
data2 = bg.boll2(data)


In [ ]:
# 3번 함수 실행
bg.boll_rtn(data2)

In [ ]:
# 클래스 생성
data = pd.read_csv("../csv/AAPL.csv",index_col= 'Date')

In [ ]:
a = bg.Invest(data,'Adj Close')

In [ ]:
a.boll('20100101','20150101')

In [ ]:
# 클래스 생성
class_a = bg.Invest(data, 'Adj Close')

In [ ]:
class_a.df

In [ ]:
class_a.boll('20100101','20150101')

In [ ]:
class_a.boll2()

In [ ]:
class_a.boll_rtn()

In [ ]:
class_a.df